In [ ]:
# Link Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reset folder with images
!rm -r images
!mkdir images

rm: cannot remove 'images': No such file or directory


In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import cv2
import os
from google.colab.patches import cv2_imshow
import urllib.request
import numpy as np
import sys

URL = "https://deploy.laion.ai/8f83b608504d46bb81708ec86e912220/dataset/"
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all("a")
links_final = []
for link in links:
    links_final.append(link.get('href'))
print(links_final[1:])
links_final.pop(0)

def download(words):
  df_captions = pd.DataFrame(columns=['image_id', 'caption'])
  number_images = 50000
  saveimage_counter = 0
  for link in links_final:
    r = requests.get(URL+link)
    open(link, "wb").write(r.content)
    print(link)
    df_parquet = pq.read_table(link)
    df = df_parquet.to_pandas()
    df = df[['TEXT', 'URL']]
    
    for row in df.itertuples():
      try:
        valid_prompt = False
        for word in words:
          if word in row.TEXT:
            valid_prompt = True
        if valid_prompt:
          url_response = urllib.request.urlopen(row.URL, timeout=5)
          img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
          img = cv2.imdecode(img_array, -1)
          img_stretch = cv2.resize(img, (64, 64))
          cv2.imwrite('/content/images/'+str(saveimage_counter)+'.jpg', img_stretch)
          new_row = {'image_id':saveimage_counter, 'caption':row.TEXT}
          #append row to the dataframe
          df_captions = df_captions.append(new_row, ignore_index=True)
          saveimage_counter += 1
        # Uncomment to see the progress
        #if saveimage_counter % 1000 == 0:
        #  print(saveimage_counter)
        if (saveimage_counter == number_images):
          return df_captions
      except:
          print("Oops!", sys.exc_info()[0], "occurred.")

['part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00012-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', 'part

In [ ]:
df_final = download(["beach", "painting", "boat"])
df_final

part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Aqua Sailboat Swing Dress - Infant, Toddler & Girls
Pro-Palestinian activists aboard one of the boats.
The 2013 Houston Boat Show features more than 1,000 powerboats, luxury cruisers, fishing boats, personal watercraft, fun activities for the whole family and so much more.   Visit HoustonBoatShow.com for more information. <span class=meta>(ABC13&#47;Blanca Beltran)</span>
Fit woman doing lunges on beach — Photo
Famous paintings of Couples: Dancers 1997
The 'KAYMAC' The boat that wouldn't go away
Boat Painting - Entrance to the Grand Canal Venice seascape boat Thomas Moran
Costa Brava beach Lloret de Mar padlocks in a row
S

In [ ]:
df_final

,image_id,caption
0,0,"Aqua Sailboat Swing Dress - Infant, Toddler & ..."
1,1,Pro-Palestinian activists aboard one of the bo...
2,2,The 2013 Houston Boat Show features more than ...
3,3,Fit woman doing lunges on beach — Photo
4,4,Famous paintings of Couples: Dancers 1997
...,...,...
49995,49995,"Preview wallpaper Switzerland, Montreux, saili..."
49996,49996,Hand painted 3 x4 dragon ball z painting
49997,49997,"sea, water, sky, horizon over water, horizon, ..."
49998,49998,a sea of ice (olsonj) Tags: morning light ice ...


In [ ]:
df_final.to_csv('captions_laion_filtered_specific.csv', index=False)

In [ ]:
!zip -r images_laion_filtered_specific.zip images 

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: images/32648.jpg (deflated 7%)
  adding: images/48775.jpg (deflated 3%)
  adding: images/34152.jpg (deflated 4%)
  adding: images/18518.jpg (deflated 4%)
  adding: images/39739.jpg (deflated 5%)
  adding: images/47551.jpg (deflated 4%)
  adding: images/34061.jpg (deflated 4%)
  adding: images/45600.jpg (deflated 5%)
  adding: images/10986.jpg (deflated 5%)
  adding: images/44878.jpg (deflated 5%)
  adding: images/10156.jpg (deflated 7%)
  adding: images/11231.jpg (deflated 4%)
  adding: images/23228.jpg (deflated 5%)
  adding: images/19945.jpg (deflated 5%)
  adding: images/4110.jpg (deflated 5%)
  adding: images/27072.jpg (deflated 4%)
  adding: images/44703.jpg (deflated 4%)
  adding: images/33953.jpg (deflated 8%)
  adding: images/36212.jpg (deflated 4%)
  adding: images/4348.jpg (deflated 5%)
  adding: images/41856.jpg (deflated 6%)
  adding: images/655.jpg (deflated 4%)
  adding: images/36928.jpg (deflated 4%)


In [ ]:
# Copy to Drive
from google.colab import drive
drive.mount('/content/drive')
!cp /content/images_laion_filtered_specific.zip /content/drive/MyDrive/Martin/fine_tuning
!cp /content/captions_laion_filtered_specific.csv /content/drive/MyDrive/Martin/fine_tuning

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
